In [34]:
# Importing required libraries
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re
import smtplib
import time
import datetime
List_DB=[]
def get_url_html_data(URL_to_Connect,heads,List_DB):
    page=requests.get(URL_to_Connect,headers=heads)
    Soup1=BeautifulSoup(page.content,"html.parser")
    Soup1=BeautifulSoup(Soup1.prettify(),"html.parser")
    url_2=Soup1.find_all("a",href=True,class_="a-link-normal a-text-normal")
    for links in (url_2):
        Q=links['href'].split("/")
        Splitter=Q[1].split("-")[0].lower()
        if ("redmi" in Splitter or "samsung" in Splitter or "xiaomi" in Splitter or "realmi" in Splitter or "oppo" in Splitter or "vivo" in Splitter or "oneplus" in Splitter or "lava" in Splitter or "nokia" in Splitter or "micromax" in Splitter or "huawei" in Splitter or "gionee" in Splitter or "asus" in Splitter or "blackberry" in Splitter):
            url_new=str(str("https://www.amazon.in")+str(links['href']))
            page=requests.get(url_new,headers=heads)
            Soup1=BeautifulSoup(page.content,"html.parser")
            Soup1=BeautifulSoup(Soup1.prettify(),"html.parser")
            data1=Soup1.find("span",id="productTitle")
            if (data1!=None):
                data1=Soup1.find("span",id="productTitle").get_text().strip()
                data1=data1.split("|")
                Req_Data=data1.pop(0)
                data1 = ' '.join(map(str, data1))
                Extras=data1.strip()
                # print(Req_Data)
                if ("(" in Req_Data):
                    Comma_Ind=Req_Data.index("(")
                    DevName=Req_Data[:Comma_Ind].strip()
                    BrandName=DevName.split(" ")[0]
                    if (")" in Req_Data):
                        Comma_End=Req_Data.index(")")
                        Desc=Req_Data[Comma_Ind+1:Comma_End]
                    else:
                        Desc=Req_Data[Comma_Ind+1:]
                    Desc=Desc.split(",")
                    # print(Desc)
                    if (len(Desc)>=3):
                        ColorName=Desc[0].strip()
                        RAMName=Desc[1].strip().split()[0]
                        StorageName=Desc[2].strip().split()[0]
                        if (len(Desc)>=4):
                            Extras2 = ' '.join(map(str, data1[3:]))
                            Extras=Extras+" "+Extras2
                        data2=Soup1.find("td",class_="a-size-base prodDetAttrValue")
                        if (data2!=None):
                            data2 = Soup1.find("td",class_="a-size-base prodDetAttrValue").get_text().strip()
                            if ("Oxygen" in data2):
                                OSName="Oxygen OS"
                            elif ("Android" in data2):
                                OSName="Android OS"
                            elif("MIUI" in data2):
                                OSName="MIUI"
                            elif("IOS" in data2):
                                OSName="IOS"
                            elif("Windows" in data2):
                                OSName="Windows OS"
                            else:
                                OSName=" "
                        else:
                            OSName=" "
                        data3=Soup1.find("td",class_="base-item-column")
                        if (data3!=None):
                            Cam_li=data3.get_text().split(" ")
                            for q in (Cam_li):
                                if (len(q)==0 or q=="\n" or q==" " or q=="\t"):
                                    Cam_li.pop(Cam_li.index(q))
                            CamName=Cam_li[0].strip()
                        else:
                            CamName=" "
                        data4=Soup1.find("span",id="priceblock_ourprice")
                        if (data4!=None):
                            PriceName=data4.get_text().strip()
                        else:
                            data4=Soup1.find("span",id="priceblock_dealprice")
                            if (data4!=None):
                                PriceName=data4.get_text().strip()
                            else:
                                PriceName=" "
                        data5=Soup1.find("span",class_="a-icon-alt")
                        if (data5!=None):
                            data5=data5.get_text().strip().split(" ")[0]
                            StarName=data5
                        else:
                            StarName=" "
                        # data6=Soup1.find_all("div",{"class":"a-expander-content reviewText review-text-content a-expander-partial-collapse-content", "data-hook":"review-collapsed"})
                        # for i in data6:
                        #     print(i.get_text().strip())
                        List_DB.append([DevName,BrandName,ColorName,RAMName,StorageName,OSName,CamName,PriceName,StarName,Extras])
                    else:
                        continue
            else:
                continue
    # print(List_DB)
    return Soup1

def go_to_next_page(Soup):
    page=requests.get(Soup,headers=heads)
    Soup1=BeautifulSoup(page.content,"html.parser")
    Soup1=BeautifulSoup(Soup1.prettify(),"html.parser")
    page=Soup1.find("ul",class_="a-pagination")
    if not page.find("li",{"class":"a-disabled a-last"}):
        url="https://www.amazon.in/"+str(page.find("li", {"class":"a-last"}).find("a")["href"])
        return url
    else:
        return

URL_to_Connect="https://www.amazon.in/s?k=Mobiles&i=electronics&rh=n%3A1389432031%2Cp_89%3AASUS%7CApple%7CBlackBerry%7CGIONEE%7CGoogle%7CHuawei%7CLava%7CLenovo%7CMicromax%7CMotorola%7CNokia%7COnePlus%7COppo%7CPanasonic%7CRedmi%7CSamsung%7CVivo%7CXiaomi%7Crealme&dc&qid=1637336532&rnid=3837712031&ref=sr_nr_p_89_21"
heads={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "Accept-Encoding": "gzip, deflate, br","DNT":"1","Connection":"close","Upgrade-Insecure-Requests": "1"}



cols_names=["Device Name","Brand","Color","RAM Memory","Storage Memory","Operating System","Camera (MP)","Price (INR)","Star Ratings (Out of 5)", "Unique Features"]
df_db=pd.DataFrame(columns=cols_names)
Kount=0
while True:
    HTML_Current=get_url_html_data(URL_to_Connect,heads,List_DB)
    Link_Next=go_to_next_page(URL_to_Connect)
    print("Page:",Kount)
    Kount+=1
    if Link_Next==None:
        break
    URL_to_Connect=Link_Next

Page: 0
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39
Page: 40
Page: 41
Page: 42
Page: 43
Page: 44
Page: 45
Page: 46
Page: 47
Page: 48
Page: 49
Page: 50
Page: 51
Page: 52
Page: 53
Page: 54
Page: 55


In [36]:
len(List_DB)

354

In [37]:
cols_names=["Device Name","Brand","Color","RAM Memory","Storage Memory","Operating System","Camera (MP)","Price (INR)","Star Ratings (Out of 5)", "Unique Features"]
Df=pd.DataFrame(columns=cols_names)

for i in range(len(List_DB)):
    Df.loc[i]=List_DB[i]

In [38]:
Df

,Device Name,Brand,Color,RAM Memory,Storage Memory,Operating System,Camera (MP),Price (INR),Star Ratings (Out of 5),Unique Features
0,Redmi 9,Redmi,Sky Blue,4GB,64GB,Android OS,13,,4.2,2.3GHz Mediatek Helio G35 Octa core Processor
1,Redmi 9A,Redmi,Nature Green,2GB,32GB,Android OS,13,"₹7,299.00",4.2,2GHz Octa-core Helio G25 Processor 5000 mAh ...
2,OPPO A31,OPPO,Mystery Black,6GB,128GB,Android OS,12,"₹12,990.00",4.2,
3,Samsung Galaxy M31,Samsung,Ocean Blue,8GB,128GB,Android OS,64,"₹16,999.00",4.2,
4,OPPO A31,OPPO,Fantasy White,6GB,128GB,Android OS,12,"₹12,990.00",4.2,
...,...,...,...,...,...,...,...,...,...,...
349,Redmi Y2,Redmi,Blue,4GB,64GB,Android OS,12,"₹10,990.00",4.2,
350,Samsung Galaxy S21,Samsung,Phantom Violet,8GB,128GB,Android OS,2400,"₹57,196.00",4.2,
351,Samsung Galaxy Note 20 Ultra 5G with Snapdrago...,Samsung,Mystic Bronze,12GB,512GB,,Front,"₹1,05,950.00",3.7,
352,Samsung Galaxy S21 Ultra 5G,Samsung,Phantom Silver,12GB,256GB,Android OS,108,"₹1,05,999.00",4.3,


In [39]:
Df.to_excel("Mobiles_DB Amazon.xlsx")